# Trabajo de caso real Arquitectura Big Data

Se trata de un conjunto de supermercados que tienen varios años de datos acumulados de productos de limpieza del hogar.
Realizan promociones de forma permanente y quieren poder predecir el volumen de ventas según las promociones que realizan.


### Importamos las librerías necesarias

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Configuración para visualizaciones
%matplotlib inline
sns.set(style="whitegrid")

### Cargamos los datos

In [2]:
locales = pd.read_csv('VMI_Locales-20190712.txt', sep='\t', encoding='ISO-8859-1', header=None)
movimientos = pd.read_csv('VMI_Movimientos-20190716.txt', sep='\t', encoding='ISO-8859-1', header=None)
promociones = pd.read_csv('VMI_Promociones-20190715.txt', sep='\t', encoding='ISO-8859-1', header=None)
prov_productos = pd.read_csv('VMI_ProvProdu-20190712.txt', sep='\t', encoding='ISO-8859-1', header=None)

In [3]:
locales.columns = ['ID Cadena', 'DescCadena', 'ID Region', 'DescRegion', 'ID Local/CD', 'DescLocal', 'Latitud', 'Longitud', 'PD']
movimientos.columns = ['ID Local/CD', 'ID producto', 'Stock en unidades', 'Venta dia anterior en unidades', 'Precio de venta unitario', 'Fecha de la transaccion', 'CD que abastece', 'RolLocal']
promociones.columns = ['ID Local/CD', 'ID producto', 'Fecha-desde', 'Fecha-Hasta', 'Total ventas en Unidades', 'Nro. Promocion', 'Fecha de la transaccion', 'Porc_Descuento']
prov_productos.columns = [
    'ID Proveedor', 
    'Razon Social Proveedor', 
    'ID Sector', 
    'Descripcion Sector', 
    'ID Seccion', 
    'Descripcion Seccion', 
    'ID Gran Familia', 
    'Descripcion Gran Familia', 
    'ID Familia', 
    'Descripcion Familia', 
    'ID Subfamilia', 
    'Descripcion Sub Familia', 
    'ID Articulo',  
    'Descripcion Articulo', 
    'ID Producto', 
    'Descripcion Producto', 
    'ID Región',
    'Columna Faltante'  # Añadimos el nombre para la columna adicional que faltaba y que no está en los doc.
]


## EDA Dataset Locales

In [4]:
locales.head()

,ID Cadena,DescCadena,ID Region,DescRegion,ID Local/CD,DescLocal,Latitud,Longitud,PD
0,1,Plaza Vea,1,Buenos Aires,9693,9693 - P Vea Villa Urquiza.,NaN,NaN,N
1,1,Plaza Vea,1,Buenos Aires,9694,9694 - P Vea JB Justo.,NaN,NaN,N
2,1,Plaza Vea,1,Buenos Aires,9695,695 - P Vea Acoyte.,0.0,0.0,N
3,1,Plaza Vea,1,Buenos Aires,9699,699 - P Vea Rivadavia.,0.0,0.0,N
4,1,Plaza Vea,1,Buenos Aires,9889,889 - Plaza Vea Carpa de liqui,NaN,NaN,N


In [5]:
locales.shape

(175, 9)

In [6]:
locales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 175 entries, 0 to 174
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   ID Cadena    175 non-null    int64  
 1   DescCadena   175 non-null    object 
 2   ID Region    175 non-null    int64  
 3   DescRegion   175 non-null    object 
 4   ID Local/CD  175 non-null    int64  
 5   DescLocal    175 non-null    object 
 6   Latitud      140 non-null    float64
 7   Longitud     140 non-null    float64
 8   PD           175 non-null    object 
dtypes: float64(2), int64(3), object(4)
memory usage: 12.4+ KB


In [7]:
locales.isnull().sum()

ID Cadena       0
DescCadena      0
ID Region       0
DescRegion      0
ID Local/CD     0
DescLocal       0
Latitud        35
Longitud       35
PD              0
dtype: int64

Anteriormente hemos observado que en la columna de Latitud y Longitud existian valores con 0.0, vamos a ver cuantos tenemos.

In [8]:
locales.eq(0.0).sum()

ID Cadena       0
DescCadena      0
ID Region       0
DescRegion      0
ID Local/CD     0
DescLocal       0
Latitud        14
Longitud       14
PD              0
dtype: int64

Observamos que hay 14 locales con la latitud y longitud en 0.0. más 35 locales con la latitud y longitud con valor NaN. De 175 locales. 

A continuación vamos a ver si existen datos duplicados.

In [9]:
# Contar duplicados basados en 'ID Local/CD'
duplicados = locales.duplicated(subset='ID Local/CD', keep=False).sum()
print(f'Número de duplicados basados en ID Local/CD: {duplicados}')


Número de duplicados basados en ID Local/CD: 0


En la columna PD he detectado que tiene valor 'N' y 'S' teniendo en cuenta la documentación, los valores nos están indicando con una N que NO es un Punto de distribución y con una S que SI es un Punto de distribución..

In [10]:
locales['PD'].unique()

array(['N', 'S'], dtype=object)

In [11]:
locales['PD'].value_counts()

PD
N    169
S      6
Name: count, dtype: int64

Después de realizar un conteo, observamos que hay 6 locales que SI son un punto de Distribución y el resto no lo son.

In [12]:
movimientos.head()

,ID Local/CD,ID producto,Stock en unidades,Venta dia anterior en unidades,Precio de venta unitario,Fecha de la transaccion,CD que abastece,RolLocal
0,242,228217,20160715,24.0,0.0,13.63,0,SI
1,242,228217,20160716,24.0,0.0,13.63,0,SI
2,242,228217,20160717,24.0,0.0,13.63,0,SI
3,242,228217,20160718,24.0,0.0,13.63,0,SI
4,242,228217,20160719,24.0,0.0,13.63,0,SI


In [13]:
promociones.head()

,ID Local/CD,ID producto,Fecha-desde,Fecha-Hasta,Total ventas en Unidades,Nro. Promocion,Fecha de la transaccion,Porc_Descuento
0,242,228217,20161023,20161030,0.98,436830,20161005,7.0
1,242,228217,20161023,20161030,0.98,436830,20161006,7.0
2,242,228217,20161023,20161030,0.98,436830,20161007,7.0
3,242,228217,20161023,20161030,0.98,436830,20161008,7.0
4,242,228217,20161023,20161030,0.98,436830,20161009,7.0


In [14]:
prov_productos.head()

,ID Proveedor,Razon Social Proveedor,ID Sector,Descripcion Sector,ID Seccion,Descripcion Seccion,ID Gran Familia,Descripcion Gran Familia,ID Familia,Descripcion Familia,ID Subfamilia,Descripcion Sub Familia,ID Articulo,Descripcion Articulo,ID Producto,Descripcion Producto,ID Región,Columna Faltante
0,1,252,Reckitt Benckiser Argentina S.A.,1,Almacn,4,Limpieza,3,Ba¤o y Hogar,4,Desodorantes de ambientes,1,Desodorantes en aerosol,27003,273920,REFILL CITRICO OIL ESCENCE AIR WICK CITRICOS ...,1,2019-07-12 11:04:07.967
1,1,252,Reckitt Benckiser Argentina S.A.,1,Almacn,4,Limpieza,3,Ba¤o y Hogar,4,Desodorantes de ambientes,1,Desodorantes en aerosol,27004,273921,FRESHMATIC AIRWICK LAVANDA REPUESTO 250.00 MLT,1,2019-07-12 11:04:07.967
2,1,252,Reckitt Benckiser Argentina S.A.,1,Almacn,4,Limpieza,3,Ba¤o y Hogar,4,Desodorantes de ambientes,1,Desodorantes en aerosol,27005,284012,AIR WICK CITRICO FULL REP.GRATIS AIR WICK AIR ...,1,2019-07-12 11:04:07.967
3,1,252,Reckitt Benckiser Argentina S.A.,1,Almacn,4,Limpieza,3,Ba¤o y Hogar,4,Desodorantes de ambientes,1,Desodorantes en aerosol,27012,359572,MAGNOLIA Y CHERRY AIR WICK REPUESTO 175.00 GR,1,2019-07-12 11:04:07.967
4,1,252,Reckitt Benckiser Argentina S.A.,1,Almacn,4,Limpieza,3,Ba¤o y Hogar,4,Desodorantes de ambientes,1,Desodorantes en aerosol,27007,415268,FRESHMATIC BOSQUE MµGICO APARATO AIR WICK LUSH...,1,2019-07-12 11:04:07.967


Observamos que la lectura de los datos ha sido incorrecta, ha puesto todos los datos en una única columna. Deberíamos de 
hacer una lectura de los datos y separarlos en las columnas correspondientes.

### Limpieza de datos